# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [2]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify

from sidecar import Sidecar


Define data path

In [3]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [4]:
region = 69300

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [4]:
buildings = gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet").to_crs(4326).reset_index()

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [5]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(buildings)

CPU times: user 1.23 s, sys: 175 ms, total: 1.4 s
Wall time: 1.38 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [6]:
sc = Sidecar(title='buildings')

Create a Map object

In [7]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [8]:
with sc:
    display(m)

List avaialable columns

In [9]:
buildings.columns

Index(['level_0', 'index', 'id', 'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu',
       'ssbCCM', 'ssbCCD', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo',
       'stbOri', 'mtbSWR', 'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi',
       'ltbIBD', 'stbCeA', 'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [10]:
# buildings.explore()

In [11]:
column = 'mtbSWR'

classifier = classify(buildings[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/mapclassify/classifiers.py:1592: UserWarning: Not enough unique values in array to form 20 classes. Setting k to 10.
  self.bins = quantile(y, k=k)


In [12]:
used_keys[column]

'shared walls ratio of buildings'

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [73]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [71]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(tess)

CPU times: user 3.35 s, sys: 376 ms, total: 3.72 s
Wall time: 3.72 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [70]:
sc = Sidecar(title='tess')

Create a Map object

In [71]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [72]:
with sc:
    display(m)

List avaialable columns

In [73]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [88]:
column = 'stcSAl'

classifier = classify(tess[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [90]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [91]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(enc)

CPU times: user 273 ms, sys: 44.1 ms, total: 317 ms
Wall time: 316 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [92]:
sc = Sidecar(title='enclosures')

Create a Map object

In [93]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [94]:
with sc:
    display(m)

List avaialable columns

In [96]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWBB'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [13]:
column = 'likWBB'

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Streets

Load data and ensure the geometries are all valid Polygons.

In [5]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [15]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 196 ms, sys: 20 ms, total: 216 ms
Wall time: 216 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [16]:
sc = Sidecar(title='streets')

Create a Map object

In [17]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [18]:
with sc:
    display(m)

List avaialable columns

In [19]:
streets.columns

Index(['id', 'geometry', 'class', 'mm_len', 'cdsbool', 'node_start',
       'node_end', 'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [20]:
streets[['sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD']].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,59661.000000,59661.000000,59661.000000,17963.000000,59661.000000,4.931400e+04,36285.000000,59661.000000,59661.000000,39799.000000
mean,138.252610,0.965622,144.597655,2409.410714,115.328171,2.265013e+05,0.069748,35.668336,0.742877,3.616847
std,274.834488,0.099060,110.857382,6024.995679,102.762411,1.868060e+05,0.100897,12.637753,0.247385,1.997335
min,0.679485,0.010614,3.741196,0.000748,0.000000,4.938551e+00,0.000093,0.000000,0.000000,0.000000
25%,45.311669,0.984742,90.177618,338.850676,31.000000,1.155624e+05,0.030137,24.076707,0.556818,2.115138
50%,83.002720,0.999621,111.584803,845.759540,97.000000,1.790765e+05,0.055390,35.602917,0.768116,3.722852
75%,143.843728,1.000000,152.808025,1990.846089,173.000000,2.735529e+05,0.090144,50.000000,1.000000,5.074082
max,11187.123379,1.000000,1813.425620,145162.535345,786.000000,2.298146e+06,9.796707,50.000000,1.000000,10.647809


In [21]:
streets['class'].unique()

array(['residential', 'secondary', 'tertiary', 'motorway', 'unclassified',
       'pedestrian', 'trunk', 'living_street', 'primary'], dtype=object)

In [22]:
assert np.allclose(streets['sdsLen'] , streets.geometry.length)

if 'mm_len' in streets.columns:
    assert np.allclose(streets['mm_len'] , streets.geometry.length)

Specify a column and pass its values into a choropleth representation within the map. 

In [26]:
column = 'sdsSPW'
used_keys[column]

'width of street profile'

In [27]:
streets[column] = streets[column].fillna(0)

In [28]:

classifier = classify(streets[column].astype(int), 'equalinterval', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [40]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [41]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 61.1 ms, sys: 17 μs, total: 61.1 ms
Wall time: 60.7 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [42]:
sc = Sidecar(title='nodes')

Create a Map object

In [43]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [44]:
with sc:
    display(m)

List avaialable columns

In [45]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [46]:
column = 'ldsCDL'
used_keys[column]

'local cul-de-sac length of street network'

In [47]:

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/mapclassify/classifiers.py:1592: UserWarning: Not enough unique values in array to form 20 classes. Setting k to 17.
  self.bins = quantile(y, k=k)
